In [4]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
import os
from tqdm import tqdm
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from hilearn import ROC_plot, CrossValidation
import csv 

def decode(a):
    if a =='A' or a =='2':
        return int(2)
    elif  a=='B' or a == '1':
        return int(1)
    elif a=='C' or a =='0':
        return int(0)
    else:
        return int(a)

preglist=os.listdir('/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/')
import random
def split_list(lst, chunk_size):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]
print(len(preglist))
for i in range(10):
    print(len(split_list(preglist,len(preglist)//10)[i]))

test0=split_list(preglist,len(preglist)//10)[0]
train0=list(set(preglist)-set(test0))
train0.extend(split_list(preglist,len(preglist)//10)[10])
test1=split_list(preglist,len(preglist)//10)[1]
train1=list(set(preglist)-set(test1))
train1.extend(split_list(preglist,len(preglist)//10)[10])
test2=split_list(preglist,len(preglist)//10)[2]
train2=list(set(preglist)-set(test2))
train2.extend(split_list(preglist,len(preglist)//10)[10])
test3=split_list(preglist,len(preglist)//10)[3]
train3=list(set(preglist)-set(test3))
train3.extend(split_list(preglist,len(preglist)//10)[10])
test4=split_list(preglist,len(preglist)//10)[4]
train4=list(set(preglist)-set(test4))
train4.extend(split_list(preglist,len(preglist)//10)[10])
test5=split_list(preglist,len(preglist)//10)[5]
train5=list(set(preglist)-set(test5))
train5.extend(split_list(preglist,len(preglist)//10)[10])
test6=split_list(preglist,len(preglist)//10)[6]
train6=list(set(preglist)-set(test6))
train6.extend(split_list(preglist,len(preglist)//10)[10])
test7=split_list(preglist,len(preglist)//10)[7]
train7=list(set(preglist)-set(test7))
train7.extend(split_list(preglist,len(preglist)//10)[10])
test8=split_list(preglist,len(preglist)//10)[8]
train8=list(set(preglist)-set(test8))
train8.extend(split_list(preglist,len(preglist)//10)[10])
test9=split_list(preglist,len(preglist)//10)[9]
train9=list(set(preglist)-set(test9))
train9.extend(split_list(preglist,len(preglist)//10)[10])


1489
148
148
148
148
148
148
148
148
148
148


In [8]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced') #predicted prob for fre grade
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced') #predicted prob for fro grade
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')#predicted prob for frefro grades

train_label=[]
ii=0
for f in train0:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2) #fresh grade
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2) #frozen grade 
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2) #fresh and frozen grades
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain) #fit model
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test0: #predict

    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)

    
    y_test=np.where((p['LBOutc1']).to_numpy()==7,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/fold0_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/fold0_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/fold0_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)


In [9]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train1:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test1:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['LBOutc1']).to_numpy()==7,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/fold1_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/fold1_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/fold1_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)


In [10]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train2:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test2:

    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)

    y_test=np.where((p['LBOutc1']).to_numpy()==7,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/fold2_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/fold2_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/fold2_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)


In [11]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train3:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test3:


    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['LBOutc1']).to_numpy()==7,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/fold3_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/fold3_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/fold3_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
  

In [12]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train4:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test4:

    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['LBOutc1']).to_numpy()==7,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/fold4_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/fold4_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/fold4_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
  

In [13]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train5:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test5:

    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['LBOutc1']).to_numpy()==7,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/fold5_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/fold5_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/fold5_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)


In [14]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train6:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test6:

    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['LBOutc1']).to_numpy()==7,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/fold6_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/fold6_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/fold6_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
   

In [15]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train7:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test7:

    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['LBOutc1']).to_numpy()==7,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/fold7_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/fold7_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/fold7_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
       

In [16]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train8:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test8:

    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['LBOutc1']).to_numpy()==7,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/fold8_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/fold8_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/fold8_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
      

In [19]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train9:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test9:

    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['LBOutc1']).to_numpy()==7,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/fold9_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/fold9_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/fold9_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
          

In [22]:
resultdir="/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/"
for i in range(0,10):
    ranking=0
    multisuccess_patients=0
    pati_list=[j for j in os.listdir(resultdir) if 'fold'+str(i) in j]
    for k in pati_list:
        p=pd.read_csv(resultdir+k)
        if p.shape[0]>1 and 1 in list(p['true']):
            multisuccess_patients=multisuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1:
                ranking=ranking+1
            else:
                continue
    print("top1accuracy for ",i,"-th fold:", ranking/multisuccess_patients)

top1accuracy for  0 -th fold: 0.38095238095238093
top1accuracy for  1 -th fold: 0.43333333333333335
top1accuracy for  2 -th fold: 0.39473684210526316
top1accuracy for  3 -th fold: 0.2916666666666667
top1accuracy for  4 -th fold: 0.2857142857142857
top1accuracy for  5 -th fold: 0.5
top1accuracy for  6 -th fold: 0.3333333333333333
top1accuracy for  7 -th fold: 0.20689655172413793
top1accuracy for  8 -th fold: 0.5
top1accuracy for  9 -th fold: 0.19047619047619047


In [24]:
resultdir="/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fre/"
for i in range(0,10):
    multisuccess_truepred=0
    multifail_truepred=0
    singlesuccess_truepred=0
    singfail_truepred=0
    multisuccess_patients=0
    multifail_patients=0
    singlesuccess_patients=0
    singfail_patients=0
    pati_list=[j for j in os.listdir(resultdir) if 'fold'+str(i) in j]
    for k in pati_list:
        p=pd.read_csv(resultdir+k)
        if p.shape[0]>1 and 1 in list(p['true']):
            multisuccess_patients=multisuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1 and p['pred_thre'][np.argmax(p['pred'])]==1:
                multisuccess_truepred=multisuccess_truepred+1
            else:
                continue
        elif p.shape[0]>1 and 1 not in list(p['true']):
            multifail_patients=multifail_patients+1
            if p['true'][np.argmax(p['pred'])]==0 and p['pred_thre'][np.argmax(p['pred'])]==0:
                multifail_truepred=multifail_truepred+1     
            else:
                continue
        elif p.shape[0]==1 and 1 not in list(p['true']):
            singfail_patients=singfail_patients+1
            if p['true'][np.argmax(p['pred'])]==0 and p['pred_thre'][np.argmax(p['pred'])]==0:
                singfail_truepred=singfail_truepred+1     
            else:
                continue
        elif p.shape[0]==1 and 1 in list(p['true']):
            singlesuccess_patients=singlesuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1 and p['pred_thre'][np.argmax(p['pred'])]==1:
                singlesuccess_truepred=singlesuccess_truepred+1     
            else:
                continue
        
        
        
    print("top1accuracy for ",i,"-th fold:", (multisuccess_truepred+multifail_truepred+singlesuccess_truepred+singfail_truepred)/(multisuccess_patients+multifail_patients+singlesuccess_patients+singfail_patients))

top1accuracy for  0 -th fold: 0.5
top1accuracy for  1 -th fold: 0.5135135135135135
top1accuracy for  2 -th fold: 0.4189189189189189
top1accuracy for  3 -th fold: 0.4594594594594595
top1accuracy for  4 -th fold: 0.5540540540540541
top1accuracy for  5 -th fold: 0.5
top1accuracy for  6 -th fold: 0.5945945945945946
top1accuracy for  7 -th fold: 0.49324324324324326
top1accuracy for  8 -th fold: 0.4391891891891892
top1accuracy for  9 -th fold: 0.581081081081081


In [20]:
resultdir="/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/"
for i in range(0,10):
    ranking=0
    multisuccess_patients=0
    pati_list=[j for j in os.listdir(resultdir) if 'fold'+str(i) in j]
    for k in pati_list:
        p=pd.read_csv(resultdir+k)
        if p.shape[0]>1 and 1 in list(p['true']):
            multisuccess_patients=multisuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1:
                ranking=ranking+1
            else:
                continue
    print("top1accuracy for ",i,"-th fold:", ranking/multisuccess_patients)

top1accuracy for  0 -th fold: 0.47619047619047616
top1accuracy for  1 -th fold: 0.6333333333333333
top1accuracy for  2 -th fold: 0.4473684210526316
top1accuracy for  3 -th fold: 0.5
top1accuracy for  4 -th fold: 0.32142857142857145
top1accuracy for  5 -th fold: 0.4166666666666667
top1accuracy for  6 -th fold: 0.3333333333333333
top1accuracy for  7 -th fold: 0.3793103448275862
top1accuracy for  8 -th fold: 0.46153846153846156
top1accuracy for  9 -th fold: 0.2857142857142857


In [25]:
resultdir="/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_fro/"
for i in range(0,10):
    multisuccess_truepred=0
    multifail_truepred=0
    singlesuccess_truepred=0
    singfail_truepred=0
    multisuccess_patients=0
    multifail_patients=0
    singlesuccess_patients=0
    singfail_patients=0
    pati_list=[j for j in os.listdir(resultdir) if 'fold'+str(i) in j]
    for k in pati_list:
        p=pd.read_csv(resultdir+k)
        if p.shape[0]>1 and 1 in list(p['true']):
            multisuccess_patients=multisuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1 and p['pred_thre'][np.argmax(p['pred'])]==1:
                multisuccess_truepred=multisuccess_truepred+1
            else:
                continue
        elif p.shape[0]>1 and 1 not in list(p['true']):
            multifail_patients=multifail_patients+1
            if p['true'][np.argmax(p['pred'])]==0 and p['pred_thre'][np.argmax(p['pred'])]==0:
                multifail_truepred=multifail_truepred+1     
            else:
                continue
        elif p.shape[0]==1 and 1 not in list(p['true']):
            singfail_patients=singfail_patients+1
            if p['true'][np.argmax(p['pred'])]==0 and p['pred_thre'][np.argmax(p['pred'])]==0:
                singfail_truepred=singfail_truepred+1     
            else:
                continue
        elif p.shape[0]==1 and 1 in list(p['true']):
            singlesuccess_patients=singlesuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1 and p['pred_thre'][np.argmax(p['pred'])]==1:
                singlesuccess_truepred=singlesuccess_truepred+1     
            else:
                continue
        
        
        
    print("top1accuracy for ",i,"-th fold:", (multisuccess_truepred+multifail_truepred+singlesuccess_truepred+singfail_truepred)/(multisuccess_patients+multifail_patients+singlesuccess_patients+singfail_patients))

top1accuracy for  0 -th fold: 0.4864864864864865
top1accuracy for  1 -th fold: 0.5405405405405406
top1accuracy for  2 -th fold: 0.46621621621621623
top1accuracy for  3 -th fold: 0.49324324324324326
top1accuracy for  4 -th fold: 0.5
top1accuracy for  5 -th fold: 0.49324324324324326
top1accuracy for  6 -th fold: 0.5
top1accuracy for  7 -th fold: 0.4527027027027027
top1accuracy for  8 -th fold: 0.4594594594594595
top1accuracy for  9 -th fold: 0.46621621621621623


In [23]:
resultdir="/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/"
for i in range(0,10):
    ranking=0
    multisuccess_patients=0
    pati_list=[j for j in os.listdir(resultdir) if 'fold'+str(i) in j]
    for k in pati_list:
        p=pd.read_csv(resultdir+k)
        if p.shape[0]>1 and 1 in list(p['true']):
            multisuccess_patients=multisuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1:
                ranking=ranking+1
            else:
                continue
    print("top1accuracy for ",i,"-th fold:", ranking/multisuccess_patients)

top1accuracy for  0 -th fold: 0.5714285714285714
top1accuracy for  1 -th fold: 0.6666666666666666
top1accuracy for  2 -th fold: 0.5
top1accuracy for  3 -th fold: 0.5
top1accuracy for  4 -th fold: 0.32142857142857145
top1accuracy for  5 -th fold: 0.4166666666666667
top1accuracy for  6 -th fold: 0.4166666666666667
top1accuracy for  7 -th fold: 0.3103448275862069
top1accuracy for  8 -th fold: 0.6153846153846154
top1accuracy for  9 -th fold: 0.3333333333333333


In [26]:
resultdir="/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/frozen/final/result/lbprob_frefro/"
for i in range(0,10):
    multisuccess_truepred=0
    multifail_truepred=0
    singlesuccess_truepred=0
    singfail_truepred=0
    multisuccess_patients=0
    multifail_patients=0
    singlesuccess_patients=0
    singfail_patients=0
    pati_list=[j for j in os.listdir(resultdir) if 'fold'+str(i) in j]
    for k in pati_list:
        p=pd.read_csv(resultdir+k)
        if p.shape[0]>1 and 1 in list(p['true']):
            multisuccess_patients=multisuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1 and p['pred_thre'][np.argmax(p['pred'])]==1:
                multisuccess_truepred=multisuccess_truepred+1
            else:
                continue
        elif p.shape[0]>1 and 1 not in list(p['true']):
            multifail_patients=multifail_patients+1
            if p['true'][np.argmax(p['pred'])]==0 and p['pred_thre'][np.argmax(p['pred'])]==0:
                multifail_truepred=multifail_truepred+1     
            else:
                continue
        elif p.shape[0]==1 and 1 not in list(p['true']):
            singfail_patients=singfail_patients+1
            if p['true'][np.argmax(p['pred'])]==0 and p['pred_thre'][np.argmax(p['pred'])]==0:
                singfail_truepred=singfail_truepred+1     
            else:
                continue
        elif p.shape[0]==1 and 1 in list(p['true']):
            singlesuccess_patients=singlesuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1 and p['pred_thre'][np.argmax(p['pred'])]==1:
                singlesuccess_truepred=singlesuccess_truepred+1     
            else:
                continue
        
        
        
    print("top1accuracy for ",i,"-th fold:", (multisuccess_truepred+multifail_truepred+singlesuccess_truepred+singfail_truepred)/(multisuccess_patients+multifail_patients+singlesuccess_patients+singfail_patients))

top1accuracy for  0 -th fold: 0.5202702702702703
top1accuracy for  1 -th fold: 0.581081081081081
top1accuracy for  2 -th fold: 0.43243243243243246
top1accuracy for  3 -th fold: 0.5202702702702703
top1accuracy for  4 -th fold: 0.46621621621621623
top1accuracy for  5 -th fold: 0.49324324324324326
top1accuracy for  6 -th fold: 0.44594594594594594
top1accuracy for  7 -th fold: 0.4391891891891892
top1accuracy for  8 -th fold: 0.41216216216216217
top1accuracy for  9 -th fold: 0.46621621621621623
